### Ejercicios Dia 1 Solución

In [1]:
import requests
import json
import pandas as pd

In [2]:
# Esta parte sirve para configurar los logs
import logging

def config_logging():
    log_format = ('%(asctime)s %(name)s.%(funcName)s[%(process)d] '
                      '%(levelname)s %(message)s')
    handlers = list()
    handlers.append(logging.StreamHandler())
    logging.basicConfig(level=logging.INFO,
                        format=log_format,
                        handlers=handlers)

config_logging()   

In [5]:
from api_handler import APIBMEHandler

In [6]:
def gen_alloc_data(ticker, alloc):
    return {'ticker': ticker,
            'alloc': alloc}

In [6]:
# Clase de un equally weighted, como parametros tiene el algo_tag, la frecuencia de rebalanceo y el mercado
class EqWAlgo:
    
    def __init__(self, algo_tag, rebal_freq=200, market='IBEX'):
        self.api_handler = APIBMEHandler(market=market, algo_tag=algo_tag)
        self.market = market
        self.algo_tag = algo_tag     
        self.rebal_freq = rebal_freq 
        self.data_close = None
    
    def insert_allocs(self):      
        data_rebalance = self.data_close.iloc[::self.rebal_freq]
        for date, prices_date in data_rebalance.iterrows():
            prices_date = prices_date.dropna()
            alloc = 1/prices_date.shape[0]
            alloc_list = [
                gen_alloc_data(tck, alloc) for tck in prices_date.index
            ]
            str_date = date.strftime('%Y-%m-%d')
            self.api_handler.post_alloc(str_date, alloc_list)
    
    def run_bt(self):
        result = self.api_handler.run_backtest()
        return result
    
    def run_algo(self):
        self.api_handler.delete_allocs()
        self.data_close = self.api_handler.get_close_data()
        self.insert_allocs()
        self.run_bt()   

In [7]:
eqw = EqWAlgo(market='DAX', rebal_freq=200, algo_tag='fcalletest_algo1')

In [8]:
# RUN ALGO: borra las allocs existentes, crea las nuevas, las inserta y ejecuta el backtesting
eqw.run_algo()

<Response [200]>


2020-08-01 11:18:36,527 api_handler.get_close_data[8849] INFO 1COV.DE
2020-08-01 11:18:36,811 api_handler.get_close_data[8849] INFO ADSGn.DE
2020-08-01 11:18:37,097 api_handler.get_close_data[8849] INFO ALVG.DE
2020-08-01 11:18:37,371 api_handler.get_close_data[8849] INFO BASFn.DE
2020-08-01 11:18:37,657 api_handler.get_close_data[8849] INFO BAYGn.DE
2020-08-01 11:18:37,935 api_handler.get_close_data[8849] INFO BEIG.DE
2020-08-01 11:18:38,185 api_handler.get_close_data[8849] INFO BMWG.DE
2020-08-01 11:18:38,464 api_handler.get_close_data[8849] INFO CBKG.DE
2020-08-01 11:18:38,765 api_handler.get_close_data[8849] INFO CECG.DE
2020-08-01 11:18:39,044 api_handler.get_close_data[8849] INFO CONG.DE
2020-08-01 11:18:39,292 api_handler.get_close_data[8849] INFO DAIGn.DE
2020-08-01 11:18:39,659 api_handler.get_close_data[8849] INFO DB11.DE^D17
2020-08-01 11:18:39,860 api_handler.get_close_data[8849] INFO DB1Gn.DE
2020-08-01 11:18:40,079 api_handler.get_close_data[8849] INFO DB1Gn.DE_0
2020-08-

{'date': '2010-01-04', 'result': True}
{'date': '2010-10-13', 'result': True}
{'date': '2011-07-26', 'result': True}
{'date': '2012-05-07', 'result': True}
{'date': '2013-02-18', 'result': True}
{'date': '2013-11-28', 'result': True}
{'date': '2014-09-16', 'result': True}
{'date': '2015-07-07', 'result': True}
{'date': '2016-04-20', 'result': True}
{'date': '2017-01-30', 'result': True}
{'date': '2017-11-14', 'result': True}
{'date': '2018-08-30', 'result': True}
{'date': '2019-06-20', 'result': True}
{'date': '2020-04-03', 'result': True}
executed
annualized_total_return    0.020387
alpha_benchmark           -0.007867
sharpe_ratio               0.146564
n_order/year               9.595983
dtype: float64
